In [1]:
#=======================================================================================================================================
# Huggingface load_dataset 으로 MLM 훈련 하기
#
# => load_dataset 으로 wiki 말뭉치를 로딩하고, 이를 토크화 시키고, 
# input_ids 에 대해 15% 확률로 [MASK]를 씌워서, 실제 모델을 훈련시키는 예제 
#
# => MLM 훈련 말뭉치는 bongsoo/kowiki20220620 사용, 평가 말뭉치는 bongsoo/bongevalsmall 사용
#
# 출처 : https://wikidocs.net/166817
#=======================================================================================================================================

import torch
import os

from tqdm.notebook import tqdm
from transformers import AutoTokenizer, BertConfig, DistilBertForMaskedLM

import sys
sys.path.append("..")
from myutils import GPU_info, seed_everything, mlogging

# wand 비활성화 
# => trainer 로 훈련시키면 기본이 wandb 활성화이므로, 비활성화 시킴
os.environ["WANDB_DISABLED"] = "true"

In [2]:
# 훈련시킬 말뭉치(사전 만들때 동일한 말뭉치 이용)
#input_corpus = "../../data11/my_corpus/my/pre-kowiki-20220620-1줄.txt"
input_corpus = "bongsoo/kowiki20220620"  # huggingface에 등록된 말뭉치 이용

# eval 말뭉치 
#eval_corpus = "../../data11/my_corpus/bong_small_eval.txt"
eval_corpus = "bongsoo/bongevalsmall"

# 기존 사전훈련된 모델
model_path = "bongsoo/mdistilbertV1"

# 기존 사전 + 추가된 사전 파일
vocab_path="bongsoo/mdistilbertV1"

# 출력
OUTPATH = '../../data11/model/distilbert/dbmc-kowiki-202206-1line/'

############################################################################
# tokenizer 관련 hyper parameter 설정
############################################################################
batch_size = 32       # batch_size
token_max_len = 128   # token_seq_len
############################################################################


device = GPU_info()
print(device)

#seed 설정
seed_everything(333)

#logging 설정
logger =  mlogging(loggername="distilbert-MLM-Trainer", logfilename="../../log/distilbert-MLM-Trainer")

True
device: cuda:0
cuda index: 0
gpu 개수: 1
graphic name: NVIDIA A30
cuda:0
logfilepath:../../log/distilbertfpt-1_2022-07-25.log


In [3]:
# tokeinzier 생성
# tokenizer 생성
# => BertTokenizer, BertTokenizerFast 둘중 사용하면됨

tokenizer = AutoTokenizer.from_pretrained(vocab_path, max_len=token_max_len, strip_accents=False, do_lower_case=False)
# fast 토크너나이즈인지 확인
print(f'{vocab_path} is_fast:{tokenizer.is_fast}')

# speical 토큰 계수 + vocab 계수 - 이미 vocab에 포함된 speical 토큰 계수(5)
vocab_size = len(tokenizer.all_special_tokens) + tokenizer.vocab_size - 5 + 1
#vocab_size = len(tokenizer.all_special_tokens) + tokenizer.vocab_size - 5
print('*special_token_size: {}, *tokenizer.vocab_size: {}'.format(len(tokenizer.all_special_tokens), tokenizer.vocab_size))
print('*vocab_size: {}'.format(vocab_size))
print('*tokenizer_len: {}'.format(len(tokenizer)))

# 모델 로딩 further pre-training 
#config = BertConfig.from_pretrained(model_path)
model = DistilBertForMaskedLM.from_pretrained(model_path, from_tf=bool(".ckpt" in model_path)) 
#model = BertForMaskedLM.from_pretrained('bert-base-multilingual-cased')    

#################################################################################
# 모델 embedding 사이즈를 tokenizer 크기 만큼 재 설정함.
# 재설정하지 않으면, 다음과 같은 에러 발생함
# CUDA error: CUBLAS_STATUS_NOT_INITIALIZED when calling `cublasCreate(handle)` CUDA 에러가 발생함
#  indexSelectLargeIndex: block: [306,0,0], thread: [0,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
#
#     해당 오류는 기존 Embedding(8002, 768, padding_idx=1) 처럼 입력 vocab 사이즈가 8002인데,
#     0~8001 사이를 초과하는 word idx 값이 들어가면 에러 발생함.
#################################################################################
model.resize_token_embeddings(len(tokenizer))

model.to(device)

Downloading:   0%|          | 0.00/578 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.17M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

bongsoo/mdistilbertV1 is_fast:True
special_token_size: 5, tokenizer.vocab_size: 146443
vocab_size: 146444
tokenizer_len: 146443


Downloading:   0%|          | 0.00/618 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/596M [00:00<?, ?B/s]

DistilBertForMaskedLM(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(146443, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): 

In [4]:
#==================================================================================================
# load_dataset을 이용하여, 훈련/평가 dataset 로딩.
#
# [로컬 데이터 파일 로딩]
# => dataset = load_dataset("text", data_files='로컬.txt')       # text 로컬 파일 로딩
# => dataset = load_dataset("csv", data_files='로컬.csv')        # csv 로컬 파일 로딩
# => dataset = load_dataset("csv", data_files='로컬.tsv', delimiter="\t")  # tsv 로컬 파일 로딩
# => dataset = load_dataset("json", data_files='로컬.json')      # json 로컬 파일 로딩
# => dataset = load_dataset("pandas", data_files='로컬.pkl')     # pickled dataframe 로컬 파일 로딩
#
# [원격 데이터 파일 로딩]
# url = "https://github.com/crux82/squad-it/raw/master/"
# data_files = {
#    "train": url + "SQuAD_it-train.json.gz",
#    "test": url + "SQuAD_it-test.json.gz",
# }
# squad_it_dataset = load_dataset("json", data_files=data_files, field="data")
#
# 출처 : https://wikidocs.net/166816
#==================================================================================================

from datasets import load_dataset

# 훈련 말뭉치 로딩
train_dataset = load_dataset(input_corpus)

# 평가 말뭉치 로딩
eval_dataset = load_dataset(eval_corpus)

# train_dataset 출력해봄
print(f"train_dataset=======================================")
print(train_dataset)
print(train_dataset['train']['text'][0:3])

print(f'\r\n\r\n')

# eval_dataset 출력해봄
print(f"eval_dataset========================================")
print(eval_dataset)
print(eval_dataset['test']['text'][0:3])

Using custom data configuration bongsoo--kowiki20220620-8b9ab42c35200e6f
Reusing dataset text (/MOCOMSYS/.cache/huggingface/datasets/text/bongsoo--kowiki20220620-8b9ab42c35200e6f/0.0.0/08f6fb1dd2dab0a18ea441c359e1d63794ea8cb53e7863e6edf8fc5655e47ec4)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration bongsoo--bongevalsmall-cfa82c943ea1c946


  0%|          | 0/1 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/40.6k [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset text downloaded and prepared to /MOCOMSYS/.cache/huggingface/datasets/text/bongsoo--bongevalsmall-cfa82c943ea1c946/0.0.0/08f6fb1dd2dab0a18ea441c359e1d63794ea8cb53e7863e6edf8fc5655e47ec4. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

train_dataset=======================================
DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 4388545
    })
})
[' 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다', ' 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다', ' 그의 별명이 "땅콩 농부" (Peanut Farmer)로 알려졌다']



eval_dataset========================================
DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 200
    })
})
['국토교통부 관계자는  실무기구에서는 각 업계별로 규제혁신형 플랫폼 택시를 하기 위해서는 어떤 규제를 풀어야 한다는 자기 안이 있어야 한다 고 말했다 ', '국책연구기관의 한 관계자는  위원회가 전문성과 대표성을 갖추고 본연의 장점을 최대한 살리기 위해서는 외부 감시와 통제가 보다 활성화돼야 한다 고 지적했다 ', '게임업계 관계자는  현장 수요보다 의료진 등 특정한 누군가의 이익을 위해 게임을 중독물질  질병으로 만들려 한다는 합리적 의심이 든다 고 꼬집었다 ']


In [5]:
# tokenizer 처리
def tokenizer_function(examples):
    result =  tokenizer(examples['text'], truncation=True, max_length=token_max_len, return_overflowing_tokens=True)
    
    # 신규 인덱스와 이전 인덱스와의 매핑 추출
    sample_map = result.pop("overflow_to_sample_mapping")
    for key, values in examples.items():
        result[key] = [values[i] for i in sample_map]
    return result


# batched=True 하면 빠른 tokenizer 이용(Rust)
%time train_dataset_fast = train_dataset.map(tokenizer_function, batched=True)

%time eval_dataset_fast = eval_dataset.map(tokenizer_function, batched=True)

'''
%time tokenized_dataset = text_dataset.map(tokenizer_function, batched=False)
print(tokenized_dataset_fast['train']['text'][0:2])
'''

  0%|          | 0/4389 [00:00<?, ?ba/s]

CPU times: user 13min 32s, sys: 18min 7s, total: 31min 39s
Wall time: 2min 30s


  0%|          | 0/1 [00:00<?, ?ba/s]

CPU times: user 162 ms, sys: 83.3 ms, total: 245 ms
Wall time: 80.8 ms


"\n%time tokenized_dataset = text_dataset.map(tokenizer_function, batched=False)\nprint(tokenized_dataset_fast['train']['text'][0:2])\n"

In [6]:
print(f"train_dataset_fast=======================================")
print(train_dataset_fast)
print(f'*fast_len:{len(train_dataset_fast["train"])}, len:{len(train_dataset["train"])}')  # fast_dataset과 dataset 길이를 비교함
print(train_dataset_fast['train'][0:2])

print(f'\r\n\r\n')

print(f"eval_dataset_fast=======================================")
print(eval_dataset_fast)
print(f'*fast_len:{len(eval_dataset_fast["test"])}, len:{len(eval_dataset["test"])}')  # fast_dataset과 dataset 길이를 비교함
print(eval_dataset_fast['test'][0:2])

train_dataset_fast=======================================
DatasetDict({
    train: Dataset({
        features: ['text', 'input_ids', 'attention_mask'],
        num_rows: 4394506
    })
})
*fast_len:4394506, len:4388545
{'text': [' 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다', ' 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다'], 'input_ids': [[101, 8924, 10003, 120044, 10530, 119860, 124092, 217, 122839, 217, 122791, 10459, 123189, 11467, 9641, 12609, 102], [101, 11087, 10954, 23545, 120044, 124440, 11261, 140574, 36251, 18347, 135699, 217, 132912, 33727, 8843, 118698, 25685, 9089, 10622, 9339, 17706, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}



eval_dataset_fast=======================================
DatasetDict({
    test: Dataset({
        features: ['text', 'input_ids', 'attention_mask'],
        num_rows: 200
    })
})
*fast_len:200, len:200
{'text': ['국토교통부 관계자는  실무기구에서는 각 업계별로 규제혁신형 플랫

In [7]:
# MLM을 위한 DataCollatorForLangunageModeling 호출
from transformers import DataCollatorForLanguageModeling

# input_ids에 대해 MLM 만들기
data_collator = DataCollatorForLanguageModeling(    # [MASK] 를 씌우는 것은 저희가 구현하지 않아도 됩니다! :-)
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

# input_ids MLM 만들고 출력 해봄
mlm_train_sample = data_collator(train_dataset_fast['train']['input_ids'][0:2])
mlm_eval_sample = data_collator(eval_dataset_fast['test']['input_ids'][0:2])

print(f"train_dataset_fast(MLM)=======================================")
print(mlm_train_sample['input_ids'][0])
print(train_dataset_fast['train'][0])

print(f'\r\n\r\n')

print(f"eval_dataset_fast(MLM)=======================================")
print(mlm_eval_sample['input_ids'][0])
print(eval_dataset_fast['test'][0])

train_dataset_fast(MLM)=======================================
tensor([   101,   8924,  10003, 120044,    103, 119860, 124092,    217, 122839,
           217, 122791,  10459,    103,  11467,   9641,  12609,    102,      0,
             0,      0,      0,      0])
{'text': ' 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다', 'input_ids': [101, 8924, 10003, 120044, 10530, 119860, 124092, 217, 122839, 217, 122791, 10459, 123189, 11467, 9641, 12609, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}



eval_dataset_fast(MLM)=======================================
tensor([   101, 122834, 133309,    103,  11018, 126397, 121481,    103,    103,
        125019,  61844,  11261, 121809, 128462,  27506, 123644, 125465,  11513,
          9952,  12310, 119797,  11018,    103, 121809,  11513,   9937, 119828,
           103, 119748,   9521,  10739,  45893,  21711,  16139,   8888, 102055,
           102,      0,      0])
{'text': '국토교통부 관계자는  실무기구에서는 각 업계별로 규제혁신형 플랫폼 택시를 하기 위해서는 어떤 규제를 풀어야

In [8]:
# 훈련 trainer 설정 
# trainer 

from transformers import Trainer, TrainingArguments

#########################################################################################
# hyper parameter 설정
#########################################################################################

epochs = 3           # epochs
#lr = 3e-5  # 학습률

total_optim_steps = len(train_dataset_fast["train"]) * epochs // batch_size   # 총 optimize(역전파) 스탭수 = 훈련dataset 계수 * epochs // 배치 크기
eval_steps=int(total_optim_steps * 0.05)           # 평가 스탭수
logging_steps=eval_steps                           # 로깅 스탭수(*평가스탭수 출력할때는 평가스탭수와 동일하게)
save_steps=int(total_optim_steps * 0.1)            # 저장 스탭수 
save_total_limit=2                                 # 마지막 2개 남기고 삭제 

print(f'*total_optim_steps: {total_optim_steps}, *eval_steps:{eval_steps}, *logging_steps:{logging_steps}, *save_steps:{save_steps}')
#########################################################################################

# cpu 사용이면 'no_cuda = True' 설정함.
no_cuda = False
if device == 'cpu':
    no_cuda = True
print(f'*no_cuda: {no_cuda}')

training_args = TrainingArguments(
    no_cuda = no_cuda,                      # GPU 사용  안함
    output_dir = OUTPATH,                   # 출력 모델 저장 경로 
    overwrite_output_dir=True,         
    num_train_epochs=epochs,                # 에폭
    #learning_rate=lr,                      # lr: 기본 5e-5
    per_gpu_train_batch_size=batch_size,    # 배치 사이즈 
    save_steps=save_steps,                  # step 수마다 모델을 저장
    save_total_limit=save_total_limit,      # 마지막 두 모델 빼고 과거 모델은 삭제
    evaluation_strategy="steps",            # 평가 전략 : steps
    eval_steps=eval_steps,                  # 평가할 스텝수
    logging_steps=logging_steps             # 로깅할 스탭수
)

# trainer로 훈련할때는 [mask] 처리된 input_ids 만 dataset으로 넘겨주면 됨.
train_dataset_fast_input_ids = train_dataset_fast['train']['input_ids']
eval_dataset_fast_input_ids = eval_dataset_fast['test']['input_ids']

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,  #MLM(Masked Language Model)
    train_dataset=train_dataset_fast_input_ids,   # 훈련 데이터셋
    eval_dataset=eval_dataset_fast_input_ids      # 평가 데이터셋
)

Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


*total_optim_steps: 411984, *eval_steps:20599, *logging_steps:20599, *save_steps:41198
*no_cuda: False


In [9]:
# 훈련 시작
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
***** Running training *****
  Num examples = 4394506
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 411987
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


Step,Training Loss,Validation Loss
20599,2.915700,3.447141
41198,2.808500,3.584274
61797,2.756800,3.524542
82396,2.724200,3.428410
102995,2.686700,3.372173
123594,2.658100,3.458271
144193,2.621800,3.154640
164792,2.581100,3.262266
185391,2.563500,3.076575
205990,2.539200,3.308743


***** Running Evaluation *****
  Num examples = 200
  Batch size = 8
***** Running Evaluation *****
  Num examples = 200
  Batch size = 8
Saving model checkpoint to ../../data11/model/distilbert/dbmc-kowiki-202206-1line/checkpoint-41198
Configuration saved in ../../data11/model/distilbert/dbmc-kowiki-202206-1line/checkpoint-41198/config.json
Model weights saved in ../../data11/model/distilbert/dbmc-kowiki-202206-1line/checkpoint-41198/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 200
  Batch size = 8
***** Running Evaluation *****
  Num examples = 200
  Batch size = 8
Saving model checkpoint to ../../data11/model/distilbert/dbmc-kowiki-202206-1line/checkpoint-82396
Configuration saved in ../../data11/model/distilbert/dbmc-kowiki-202206-1line/checkpoint-82396/config.json
Model weights saved in ../../data11/model/distilbert/dbmc-kowiki-202206-1line/checkpoint-82396/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 200
  Batch size = 8
***** Running Eva

TrainOutput(global_step=411987, training_loss=2.5472607226973003, metrics={'train_runtime': 50330.347, 'train_samples_per_second': 261.94, 'train_steps_per_second': 8.186, 'total_flos': 2.595003351686187e+17, 'train_loss': 2.5472607226973003, 'epoch': 3.0})

In [10]:
# 모델 저장
### 전체모델 저장
TMP_OUT_PATH = '../../data11/model/distilbert/mydistilbert2/7/'
os.makedirs(TMP_OUT_PATH, exist_ok=True)
#torch.save(model, OUTPATH + 'pytorch_model.bin') 
# save_pretrained 로 저장하면 config.json, pytorch_model.bin 2개의 파일이 생성됨
model.save_pretrained(TMP_OUT_PATH)

# tokeinizer 파일 저장(vocab)
VOCAB_PATH = TMP_OUT_PATH
tokenizer.save_pretrained(VOCAB_PATH)
print(f'==> save_model : {TMP_OUT_PATH}')

Configuration saved in ../../data11/model/distilbert/mydistilbert2/7/config.json
Model weights saved in ../../data11/model/distilbert/mydistilbert2/7/pytorch_model.bin
tokenizer config file saved in ../../data11/model/distilbert/mydistilbert2/7/tokenizer_config.json
Special tokens file saved in ../../data11/model/distilbert/mydistilbert2/7/special_tokens_map.json


==> save_model : ../../data11/model/distilbert/mydistilbert2/7/
